# Install xmltodict (Not Default with ArcGIS Pro)

In [ ]:
#Install xmltodict
try:
    %pip install xmltodict

except:
    !pip install xmltodict

# Define Functions

In [ ]:
import os.path
import xmltodict

#Define functions
def arcgis_metadata_dict(items):
    """
    Formats items from ArcGIS Portal into a dictionary of metadata.
    
    Parameters:
    * `items`: A list of items from ArcGIS Portal such as from the `gis.content.search` function.
    """
    metadata = {}
    for item in items:
        #Read XML Data to Dictionaries
        if "Error: Metadata file does not exist or is inaccessible" not in item.metadata:
            if "metadata.xml" in item.metadata:
                with open(item.metadata, 'rb') as file:
                    metadata[item.id] = xmltodict.parse(file)['metadata']
                    
        #If no Metadata Exists, Assign Null
        if item.id not in metadata:
            metadata[item.id] = None
            
    return metadata
    
def format_metadata_dict(input_dict: dict, fields: list, skip_no_data = True):
    """
    Pulls data from metadata fields and exports it to a flattened table format.
    
    Parameters:
    * `input_dict`: Dictionary outputted from `arcgis_metadata_dict`.
    * `fields`: Fields to export from the dictionary mapped to new names to use. 
    These are written with slashes between for each level. Ex: "Esri/CreaDate" 
    to get creation date.
    * `skip_no_data`: If items with no metadata file should be skipped.
        * Default: `None`
    """
    #Convert Fields to a List
    if not isinstance(fields, dict):
        raise TypeError('Fields must be a dictionary.')
    fields_map = fields
    fields = [field.split('/') for field in fields]

    #Iterate Through Items
    out_table = []
    for item in input_dict:
        #Get Dictionary for Specific Item
        item_dict = input_dict[item]
        table_row = {'id':item}
        
        #if No Data, Skip or Assign None to All Fields
        if item_dict is None:
            if skip_no_data is True:
                continue
                
            for field in list(fields_map.keys()):
                table_row[fields_map[field]] = None
            
            #Add Row to Table
            out_table.append(table_row)
            continue
        
        for index, field in enumerate(fields):
            #Traverse through Dictionaries to Get Values
            for level in field:
                try:
                    try:
                        out_field_data
                        out_field_data = out_field_data[level]
                    except:
                        out_field_data = item_dict[level]
                except:
                    out_field_data = None
            
            #Add Value to Table Row
            table_row[fields_map[list(fields_map.keys())[index]]] = out_field_data
            del out_field_data
        
        #Add Row to Table
        out_table.append(table_row)

    return out_table

# Usage Example
`dict_fields`: A dictionary of fields to be pulled from XML data. The keys will represent the actual fields, while the values represent the field names they will be assigned to in the table. Fields in XML have to be written with "levels". For each indent in the file, a new field level can be added with a "/".

`csv_file`: The CSV file the table will be saved to.

In [ ]:
#Parameters
dict_fields = {'Esri/CreaDate':'CreateDate'}
csv_file = 'C://path/to/your/file.csv'

In [ ]:
from arcgis import GIS
import pandas as pd

#Login to ArcGIS Portal
gis = GIS('pro')

#Get Items
##This only gives you 500 items
items = gis.content.search('*', max_items = 500)

#Convert XML Files to Nested Dictionaries
metadata_dict = arcgis_metadata_dict(items)

#Pull Data from Dictionaries to Make a Flat Table
metadata_table = format_metadata_dict(input_dict = metadata_dict, fields = dict_fields, skip_no_data = True)

#Conver to DataFrame
metadata_df = pd.DataFrame(metadata_table)

#Write to CSV File (Uncomment to Use)
pd.metadata_df.to_csv(csv_file, index = False)

#View DataFrame
metadata_df